# A value history player widget

Consider a value widget `w`.
Player(`w`) wraps this widget to maintain a history of all the values that have been assigned to it.
In addition, it adds controls analogous to those of audio/video/... players to
replay that history.

In [1]:
from ipywidgets import HBox, VBox, Button, Text, Layout, Output, IntSlider, Label
import traitlets
import threading, queue, os, time

Current bug: when playing at the end while new values are being produced, the current clock can't be cancelled
GIL issue?

In [2]:
class Clock(threading.Thread):
    """
    Execute a command every x seconds
    """
    def __init__(self, delay, command):
        self.cancelled = False
        self.delay = delay
        self.command = command
        threading.Thread.__init__(self)

    def run(self):
        while not self.cancelled:
            print("Running command")
            if self.command():
                time.sleep(self.delay)
            else:
                self.cancel()
            
    def cancel(self):
        print("canceling")
        self.cancelled = True

In [3]:
class PlayerModel:
    def __init__(self, view, controler):
        # Invariant: -1 <= time <= len(history) -1
        # If time <> -1, history[time] is the currently displayed value
        self._view = view
        self._controler = controler
        self._clock = None
        self._finished = False
        self.reset()
        self.play()
    
    def reset(self):
        self.stop()
        self._new_values = queue.Queue(maxsize=1)
        self._history = []
        self._time = -1
        self._direction = 1
        self._speed = 1

    def update(self):
        self._view.value = self._history[self._time]
        self._controler._frame.value = str(self._time)
   
    def set_value(self, value):
        self._new_values.put(value)
    
    def stop(self):
        #print("stop", self._clock)
        if self._clock:
            self._clock.cancel()
            self._clock = None

    def begin(self):
        self.stop()
        assert self._history
        self._time = 0
        self.update()

    def end(self):
        # Could trigger a clock if not finished
        self.stop()
        self._time = len(self._history) - 1
        self.update()

    def step_backward(self):
        self.stop()
        self.step(direction=-1)

    def step_forward(self):
        self.stop()
        self.step(direction=1)

    def step(self, direction=1):
        if direction < 0:
            if self._time > 0:
                self._time -= 1;
                self.update()
                return True
        else:
            assert self._time < len(self._history)
            while self._time == len(self._history) - 1:
                if self._finished:
                    return False
                try:
                    self._history.append(self._new_values.get(timeout=1))
                    break
                except queue.Empty:
                    pass
            assert self._time < len(self._history) - 1
            self._time += 1
            self.update()
            return True

    def play(self, direction=None):
        self.stop()
        if direction is None:
            direction = self._direction
        else:
            self._direction = direction
        self._clock = Clock(abs(1./self._speed), lambda: self.step(direction))
        self._clock.start()
        
    def set_speed(self, speed):
        self._speed = 2**speed
        if self._clock: # Restart the clock
            self.play()

In [4]:
class Player(VBox):
    value = traitlets.Any()
    def __init__(self, view):
        model = PlayerModel(view, self)
        self._model = model
        
        layout  = Layout(width="initial")
        step_backward = Button(tooltip="Step backward", icon="step-backward", layout=layout)
        step_forward  = Button(tooltip="Step forward",  icon="step-forward",  layout=layout)
        begin         = Button(tooltip="Begin",         icon="fast-backward", layout=layout)
        end           = Button(tooltip="End",           icon="fast-forward",  layout=layout)
        backward      = Button(tooltip="Play backward", icon="backward",      layout=layout)
        play          = Button(tooltip="Play",          icon="play",          layout=layout)
        pause         = Button(tooltip="Pause",         icon="pause",         layout=layout)
        
        step_backward.on_click(lambda button: model.step_backward())
        step_forward .on_click(lambda button: model.step_forward())
        begin        .on_click(lambda button: model.begin())
        end          .on_click(lambda button: model.end())
        play         .on_click(lambda button: model.play(1))
        backward     .on_click(lambda button: model.play(-1))
        pause        .on_click(lambda button: model.stop())
        frame_label = Label("frame: ")
        frame = Label("")
        self._frame = frame
        speed_label = Label("speed: ")
        speed_slider = IntSlider(0,0,10,1)
        speed_slider.observe(lambda slider: model.set_speed(speed_slider.value))
        controller = HBox([begin, backward, step_backward, pause, play, step_forward, end, frame_label, frame, speed_label, speed_slider])
        VBox.__init__(self, [view, controller])
    
    @traitlets.observe("value")
    def value_changed(self, change):
        self._model.set_value(change.new)

In [5]:
w = Text()
p = Player(w)
p

Running command


Player(children=(Text(value=''), HBox(children=(Button(icon='fast-backward', layout=Layout(width='initial'), s…

In [ ]:
p.value = "9"

In [ ]:
for i in range(10):
    #
    print(i)
    p.value = str(i)

0
1
2


In [13]:
p = Player(Text())
def mysort(l):
    p.value = str(l)
    for i in range(len(l)):
        for j in range(i, 0, -1):
            if l[j-1] <= l[j]:
                break
            l[j-1], l[j] = l[j], l[j-1]
            p.value = str(l)
    return l
p

stop None
stop None


Player(children=(Text(value=''), HBox(children=(Button(icon='fast-backward', layout=Layout(width='initial'), s…

stop <Clock(Thread-8, started 140167373375232)>
stop <Clock(Thread-9, started 140167364982528)>
stop <Clock(Thread-10, started 140167356589824)>
stop <Clock(Thread-11, started 140167348197120)>
stop <Clock(Thread-12, started 140166861682432)>
stop <Clock(Thread-13, started 140166853289728)>
stop <Clock(Thread-14, started 140166844897024)>
stop <Clock(Thread-15, stopped 140166861682432)>
stop <Clock(Thread-16, started 140166844897024)>
stop <Clock(Thread-17, started 140166861682432)>
stop <Clock(Thread-18, started 140166853289728)>
stop <Clock(Thread-19, started 140167348197120)>


In [14]:
mysort([6,5,4,3,2,1])

[1, 2, 3, 4, 5, 6]

In [18]:
p._model._clock